# Reinforcement Learning
## Formative Assessment: Value Iteration
In this formative exercise, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but closely related) Markov Decision Processes. For your reference, the pseudo-code for the Value Iteration algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 2018, pp. 83).

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The symbol $r$ represents the immediate reward on transition from state $s$ to the next state $s'$ via action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 320px;" align="left" caption="Figure 1"/>

The two problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this kind of environment from the content you have seen so far in this unit. The grid squares in the figure are numbered as shown. In all exercises, the following are true: 

**Actions available:** The agent has four possible actions in each grid square. These are *west*, *north*, *south*, and *east*. If the direction of movement is blocked by a wall (for example, if the agent executes action south in grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold (from a neighbouring grid square), the agent collects the gold. Note that, in order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square.

**Hitting the bomb:** On arrival at a grid square that contains a bomb (from a neighbouring grid square), the agent activates the bomb. 

**Terminal states:** The game terminates when **all** gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to $1 \times 10 ^{-10}$. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold (from a neighbouring grid square) is $-1 + 10 = +9$. 

## Exercise 1: Deterministic Environment (0 Marks)

In this exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment.

Compute the optimal policy using Value Iteration. 

Your need to produce two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index $i$ representing grid cell $i$ (see figure above). Both arrays should be accessible in the "solution cell" below!

The array `policy` should be a numpy array of strings that specifies an optimal action at each grid location. Please use the abbreviations `"n"`, `"e"`, `"s"`, and `"w"` for the four actions. As an example, the value of `policy` at index `0` needs to give `"n"`, if _north_ is an optimal action in cell 0. The policy for a terminal state can be any action. If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value under the optimal policy).

In [1]:
import numpy as np

action_names = [
    'North',
    'East',
    'South',
    'West',
]

actions = [
    np.array([1, 0]), # North
    np.array([0, 1]), # East
    np.array([-1, 0]),# South
    np.array([0, -1]),# West
]

action_abrev = [
    'n',
    'e',
    's',
    'w',
]


def print_state_dynamics(transition_dynamics, state):
    dynamics = transition_dynamics[state]
    
    print('State {}'.format(state))
    
    for action, action_dynamics in dynamics.items():
        print('\t{}'.format(action_names[action]))
        for actual_action, action_probs in action_dynamics.items():
            print('\t\t{} - {}'.format(action_names[actual_action], action_probs))

def print_transition_dynamics(transition_dynamics):
    """
    Use this function if you would like to visualise the transition dynamics.
    
    Call this function with the transition dynamics as a parameter.
    """
    for state, dynamics in transition_dynamics.items():
        print_state_dynamics(transition_dynamics, state)

In [2]:
def create_transition_dynamics(num_states, gold_states, terminal_states, bomb_states, stochasticity=1):
    """
    Function to create the transition dynamics for each of the different gridworld problems.
    Returns a dictionary of the following format.
    'state': [
        'north_action': [
            'north_action': (probability, new_state, reward)
            'east_action':  (probability, new_state, reward)
            'south_action': (probability, new_state, reward)
            'west_action':  (probability, new_state, reward)
        ],
        'east_action': [
            'north_action': (probability, new_state, reward)
            'east_action':  (probability, new_state, reward)
            'south_action': (probability, new_state, reward)
            'west_action':  (probability, new_state, reward)
        ],
        ...
    ]
    
    The dictionary has one entry for every state. Each state has the four actions an agent could choose, 
    and each action has each of the 4 actions that could actually be taken if the environment is stocastic.
    This means the dicationary can be used to lookup the transition dynamics for a given state, action pair. 
    Essentially, this function implements the p(s',r|s,a).
    
    We can lookup the values like so:
    
        transition_dynamics[state][action]
        
    to get the list of possible transitions this can result in and their associated probabilities:
        
        'north_action': (probability, new_state, reward)
        'east_action':  (probability, new_state, reward)
        'south_action': (probability, new_state, reward)
        'west_action':  (probability, new_state, reward)
    
    This allows use to iterate over each of the potential outcomes of taking an action in a state and weight
    them by their probability of occuring.
    """
    action_reward = -1
    gold_reward = 10
    bomb_reward = -10

    transition_dynamics = {}
    random_action_probability = (1-stochasticity)/len(actions)

    for s in range(num_states):
        # The state multiplier is used to keep the equations consistent in the case where we
        # have more than one gold. We use the state_multiplier to move the boards found out
        # states 25:50 and 50:75 back to 0:25 for ease/consistency of calculation.
        state_multiplier = s//25
        state_offset = state_multiplier * 25
        offset_s = s - state_offset
        x = int(offset_s/5)
        y = int(offset_s - (x * 5))
        pos = np.array([x, y])

        action_transitions = {}
        
        # For each action in a state, calculate all of its possible outcomes
        for action in range(len(actions)):
            action_transition_dynamics = {}
            
            # When taking an action iterate over each of the outcomes that could actually happen.
            # Use this for stocastic transitions.
            for final_direction in range(len(actions)):
                
                # Calculate the new state the action results in and clip the outcome to between 0 and 4.
                # The clipping is to stop the transitions traversing of the edge of the board.
                direction = actions[final_direction]
                new_position = pos + direction
                new_position = np.clip(new_position, 0, 4)

                action_probability = random_action_probability
                if action == final_direction:
                    action_probability += stochasticity
                
                # Convert x,y positions back to a single state id.
                new_state = (new_position[0] * 5) + new_position[1]
                
                # Caculate the reward for the transition that the state, action and new_state give.
                # We need to translate the states back by their offset we calculated at the beginning.
                # This allows us to still use bomb at state 67 instead of keeping track of which board
                # configuration we are in for the case where we have 2 golds.
                reward = action_reward
                if new_state + state_offset in bomb_states:
                    reward += bomb_reward
                elif new_state + state_offset in gold_states and s != new_state:
                    reward += gold_reward
                
                # If we are dealing with the two gold scenario check if we are in a gold state on the board
                # with 2 gold pieces and adject their future states to be on the new boards.
                # See the first few slides of the lecture for a visual explanation of this.
                if num_states > 25:
                    if new_state + state_offset == 12:
                        new_state = 12 + 25

                    if new_state + state_offset == 23 and s != 23:
                        new_state = 23 + 50

                new_state += state_offset
                
                # Create and store the transition
                transition = (round(action_probability, 2), new_state, reward)

                action_transition_dynamics[final_direction] = transition
            action_transitions[action] = action_transition_dynamics
        transition_dynamics[s] = action_transitions
    return transition_dynamics

In [9]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm.

num_states = 25

gold_states = [23]
terminal_states = [18, 23]
bomb_states = [18]

stochasticity = 1

theta = 1e-10
gamma = 1 # Discount Factor

# Get the transition dynamics for the current problem.
transition_dynamics = create_transition_dynamics(num_states, gold_states, terminal_states, bomb_states, stochasticity)


# Value iteration begins from here.
v = np.zeros(num_states)

delta = np.inf

while delta > theta:
    delta = 0
    
    for s in range(num_states):
        if s in terminal_states:
            continue

        old_v = v[s]
        new_vs = []
        for a in range(len(actions)):
            new_v = 0
            transitions = transition_dynamics[s][a]
            for a_a, p in transitions.items():
                # probability * (r + γV(S'))
                new_v += (p[0] * (p[2] + gamma * v[p[1]]))
            new_vs.append(new_v)
        new_v = max(new_vs)
        delta = max(delta, abs(new_v - old_v))
        v[s] = new_v

policy_states = 25
policy = ['' for _ in range(policy_states)]

# Perform one-step look ahead to derive the policy from the values.
for s in range(policy_states):
    action_values = np.zeros(len(actions))
    for a in range(len(actions)):
        for a_a, p in transition_dynamics[s][a].items():
            action_values[a] += (p[0] * (p[2]+ v[p[1]]))
    policy[s] = action_abrev[np.argmax(action_values)]

policy = np.array(policy)


**Example Test Cell**

In the code cell below, we have provided an example of the type of test code that we will use to mark your work. In these tests, we first check that your `policy` and `v` variables are of the correct type, and then check that their values match the solution. For the remaining exercises, the answers will be hidden from you until the model answers are released.

In [10]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 1 is tested here.

import numpy as np

# We're giving you the solution values for Exercise 1, but not telling you how to compute them!
solution_values = [3.0, 4.0, 5.0, 4.0, 5.0,
                   4.0, 5.0, 6.0, 5.0, 6.0,
                   5.0, 6.0, 7.0, 6.0, 7.0,
                   6.0, 7.0, 8.0, 0.0, 8.0,
                   7.0, 8.0, 9.0, 0.0, 9.0]
solution_values = np.array(solution_values)

# We're giving you the solution policy for Exercise 1, but not telling you how to compute it!
solution_policy = [
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'e', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'e', 'e', 'e', 'n', 'w',]
solution_policy = np.array(solution_policy)

# Check that policy and v are numpy arrays.
assert(isinstance(policy, np.ndarray))
assert(isinstance(v, np.ndarray))

# Check correct shapes of numpy arrays.
assert(policy.shape == (25, ))
assert(v.shape == (25, ))

# Check whether the numpy arrays have the correct data types.
assert(np.issubdtype(policy.dtype, np.unicode_)) # policy.dtype should be '<U1'
assert(np.issubdtype(v.dtype, np.float64))

# Check whether policy contains only "n", "w", "s", or "e" values.
assert(np.all(np.isin(policy, np.array(["n", "w", "s", "e"]))))

# Print student's solution and true solution for easier comparison / spotting of errors.
print("Student's policy:")
print(np.flip(policy.reshape((5, 5)), 0))
print("Solution policy:")
print(np.flip(solution_policy.reshape((5, 5)), 0))
print("\n")
print("Student's v:")
print(np.flip(v.reshape((5, 5)), 0))
print("Solution v:")
print(np.flip(solution_values.reshape((5, 5)), 0))

# Compare state values for non-terminal states.
states_to_check = np.delete(np.arange(25), np.array([18, 23]))
incorrect_error_message = "[Marking Advice] State values incorrect."
np.testing.assert_array_almost_equal(v[states_to_check], solution_values[states_to_check], decimal=3, err_msg=incorrect_error_message)

# Compare policy (only on states that have a single optimal direction).
states_to_check =  np.array([4, 9, 14, 17, 19, 20, 22, 24])
incorrect_error_message = "[Marking Advice] Policy incorrect."
np.testing.assert_array_equal(policy[states_to_check], solution_policy[states_to_check], err_msg=incorrect_error_message)

# Compare state values for terminal states.
states_to_check =  np.array([18, 23])
incorrect_error_message = "[Marking Advice] Value of terminal state is not zero."
np.testing.assert_array_equal(v[states_to_check], solution_values[states_to_check], err_msg=incorrect_error_message)

Student's policy:
[['e' 'e' 'e' 'e' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Solution policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]


Student's v:
[[ 3.96      6.2       9.        0.        9.      ]
 [ 2.168     3.96      6.2       0.        6.2     ]
 [ 0.7344    2.168     3.96      2.168     3.96    ]
 [-0.41248   0.7344    2.168     0.7344    2.168   ]
 [-1.329984 -0.41248   0.7344   -0.41248   0.7344  ]]
Solution v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]


AssertionError: 
Arrays are not almost equal to 3 decimals
[Marking Advice] State values incorrect.
Mismatched elements: 21 / 23 (91.3%)
Max absolute difference: 4.41248
Max relative difference: 1.443328
 x: array([-1.33 , -0.412,  0.734, -0.412,  0.734, -0.412,  0.734,  2.168,
        0.734,  2.168,  0.734,  2.168,  3.96 ,  2.168,  3.96 ,  2.168,
        3.96 ,  6.2  ,  6.2  ,  3.96 ,  6.2  ,  9.   ,  9.   ])
 y: array([3., 4., 5., 4., 5., 4., 5., 6., 5., 6., 5., 6., 7., 6., 7., 6., 7.,
       8., 8., 7., 8., 9., 9.])

## Exercise 2: Stochastic Environment

In this exercise, we introduce stochasticity into the environment. Now, the agent is not always able to execute its actions as intended.

With probability 0.8, the agent moves as intended. However, with probability 0.2, it moves in a random direction.

For example, from grid square 0, if the agent tries to move north, with probability 0.8 the action will work as intended. But with probability 0.2, the agent's motor control system will move it in a random direction (including north). So, it will randomly try to move west, east, north or south with probability 0.05 each. Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [3]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm.
num_states = 25

gold_states = [23]
terminal_states = [18, 23]
bomb_states = [18]

stochasticity = 0.8

theta = 1e-10
gamma = 1 # Discount Factor

transition_dynamics = create_transition_dynamics(num_states, gold_states, terminal_states, bomb_states, stochasticity)

v = np.zeros(num_states)

delta = np.inf

while delta > theta:
    delta = 0
    
    for s in range(num_states):
        if s in terminal_states:
            continue

        old_v = v[s]
        new_vs = []
        for a in range(len(actions)):
            new_v = 0
            transitions = transition_dynamics[s][a]
            for a_a, p in transitions.items():
                # probability * (r + γV(S'))
                new_v += (p[0] * (p[2] + gamma * v[p[1]]))
            new_vs.append(new_v)
        new_v = max(new_vs)
        delta = max(delta, abs(new_v - old_v))
        v[s] = new_v

policy_states = 25
policy = ['' for _ in range(policy_states)]

for s in range(policy_states):
    action_values = np.zeros(len(actions))
    for a in range(len(actions)):
        for a_a, p in transition_dynamics[s][a].items():
            action_values[a] += (p[0] * (p[2]+ v[p[1]]))
    policy[s] = action_abrev[np.argmax(action_values)]

v = np.around(v, 2)
policy = np.array(policy)

In [4]:
# DO NOT DELETE OR MODIFY THIS CELL!

print("Student's v:")
print(np.flip(v.reshape((5, 5)), 0))
print("\n")
print("Student's policy:")
print(np.flip(policy.reshape((5, 5)), 0))

Student's v:
[[6.04 7.29 8.61 0.   8.69]
 [4.86 5.99 6.37 0.   6.47]
 [3.68 4.7  4.99 3.22 5.1 ]
 [2.49 3.41 3.67 2.64 3.79]
 [1.36 2.2  2.43 1.57 2.55]]


Student's policy:
[['e' 'e' 'e' 'e' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']]


## Exercise 3: Stochastic Environment with Two Pieces of Gold (6 Marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this exercise, we have modified the environment presented in exercise 2. A second piece of gold has been placed on grid square 12, and a terminal state is reached only when **both** pieces of gold have been collected, or when the bomb is activated.

Compute the optimal policy for this altered environment using Value Iteration.

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in the previous exercises. Each value in these arrays should specify the expected return and optimal policy at the corresponding grid sqaure **before any pieces of gold are collected or a bomb is activated.** 

Hint: You will need to change your state representation in order to account for the additional piece of gold.

In [7]:
# Please write your code for Exercise 3 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm.

# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm.

num_states = 75

gold_states = [12, 23, 48, 62]
terminal_states = [18, 43, 48, 62, 68]
bomb_states = [18, 43, 68]

stochasticity = 0.8

theta = 1e-10
gamma = 1 # Discount Factor

transition_dynamics = create_transition_dynamics(num_states, gold_states, terminal_states, bomb_states, stochasticity)

# print_transition_dynamics(transition_dynamics)

v = np.zeros(num_states)

delta = np.inf

while delta > theta:
    delta = 0
    
    for s in range(num_states):
        if s in terminal_states:
            continue

        old_v = v[s]
        new_vs = []
        for a in range(len(actions)):
            new_v = 0
            transitions = transition_dynamics[s][a]
            for a_a, p in transitions.items():
                # probability * (r + γV(S'))
                new_v += (p[0] * (p[2] + gamma * v[p[1]]))
            new_vs.append(new_v)
        new_v = max(new_vs)
        delta = max(delta, abs(new_v - old_v))
        v[s] = new_v

policy_states = 25
policy = ['' for _ in range(policy_states)]

for s in range(policy_states):
    action_values = np.zeros(len(actions))
    for a in range(len(actions)):
        for a_a, p in transition_dynamics[s][a].items():
            action_values[a] += (p[0] * (p[2]+ v[p[1]]))
    policy[s] = action_abrev[np.argmax(action_values)]

policy = np.array(policy)

v = v[:25]
v = np.around(v, 2)
policy = policy[:25]

# YOUR CODE HERE

In [8]:
# DO NOT DELETE OR MODIFY THIS CELL!

print("Student's v:")
print(np.flip(v.reshape((5, 5)), 0))
print("\n")
print("Student's policy:")
print(np.flip(policy.reshape((5, 5)), 0))

Student's v:
[[10.65 11.8  13.01 10.74 12.97]
 [11.19 12.33 12.51  0.   10.62]
 [12.29 13.59 12.48 12.42 11.2 ]
 [11.18 12.35 13.59 12.28 11.05]
 [10.06 11.17 12.28 11.11  9.99]]


Student's policy:
[['e' 'e' 'e' 'w' 'w']
 ['e' 's' 's' 'n' 'n']
 ['e' 'e' 'w' 'w' 'w']
 ['e' 'n' 'n' 'w' 'w']
 ['n' 'n' 'n' 'n' 'w']]
